<a href="https://colab.research.google.com/github/Teasotea/DialogSystem/blob/main/ConversationalAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install transformers

In [23]:
!pip install nltk

In [84]:
import numpy as np
import pandas as pd
import time
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import nltk
from nltk.stem.lancaster import LancasterStemmer
from sklearn.svm import OneClassSVM

In [85]:
stemmer = LancasterStemmer()

# Part I: Greeting Classification

In [86]:
training_data = []
greetings =list(dict.fromkeys(['hi', "hola", 'hey', 'hello','morning', 'evening', 'good day', 'good morning', 'greetings', 'howdy', 'welcome', 'bonjour',
             'buenas noches', 'buenos dias', 'salutation', 'salut', 'hail', 'salaam', 'aloha', 'ciao', 'good wishes',
             'aloha', 'yoo-hoo', 'yawp', 'It’s always a pleasure to see you.', 'oh', 'ave', 'yo', 'hi there', 
             "hi", "hello", "hey", "helloo", "hellooo", "g morining", "gmorning", "good morning", "morning", "good day", "good afternoon", "good evening", "greetings", "greeting", 
             "good to see you", "its good seeing you", "g’day", "howdy"]))

for i in greetings:
  training_data.append({"class":"greeting", "sentence":i})
# training_data.append({"class":"other", "sentence":'word'})
  
greet_df = pd.DataFrame(training_data)
greet_df

,class,sentence
0,greeting,hi
1,greeting,hola
2,greeting,hey
3,greeting,hello
4,greeting,morning
5,greeting,evening
6,greeting,good day
7,greeting,good morning
8,greeting,greetings
9,greeting,howdy


In [87]:
greetings_test = ['hi, there!','psst, can you tell me, which algorithm would work best if I have imbalanced dataset?','What\'s your favourite music?','Hello? Somebody is here?', 'people often say welcome to guests','How many answers can you provide on this topic?',"hola, hola!", 'tell me about yourself','are you a bot','which movie would ypu suggest to watch this evening?', 'which laptop is the best for Data Science?', 'What about language generation task?','hey, bot:)', 'water']
testy = np.array([ 1, -1, -1, 1, -1, -1, 1, -1,  -1, -1, -1,  -1,  1, -1])

### One-Class Classification with SVM

In [88]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score

In [89]:
num_words = 1000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

In [90]:
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer.fit_on_texts(greetings)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(greetings)

# Get max training sequence length
maxlen = max([len(x) for x in train_sequences])

# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

# Output the results of our work
print("Word index:\n", word_index)
print("\nTraining sequences:\n", train_sequences)
# print("\nPadded training sequences:\n", train_padded)
print("\nPadded training shape:", train_padded.shape)
print("Training sequences data type:", type(train_sequences))
print("Padded Training sequences data type:", type(train_padded))

test_sequences = tokenizer.texts_to_sequences(greetings_test)
test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

print("Testing sequences:\n", test_sequences)
print("\nPadded testing sequences:\n", test_padded)
print("\nPadded testing shape:",test_padded.shape)

for x, y in zip(greetings_test, test_padded):
  print('{} -> {}'.format(x, y))

Word index:
 {'<UNK>': 1, 'good': 2, 'you': 3, 'hi': 4, 'morning': 5, 'evening': 6, 'to': 7, 'see': 8, 'hola': 9, 'hey': 10, 'hello': 11, 'day': 12, 'greetings': 13, 'howdy': 14, 'welcome': 15, 'bonjour': 16, 'buenas': 17, 'noches': 18, 'buenos': 19, 'dias': 20, 'salutation': 21, 'salut': 22, 'hail': 23, 'salaam': 24, 'aloha': 25, 'ciao': 26, 'wishes': 27, 'yoo': 28, 'hoo': 29, 'yawp': 30, 'it’s': 31, 'always': 32, 'a': 33, 'pleasure': 34, 'oh': 35, 'ave': 36, 'yo': 37, 'there': 38, 'helloo': 39, 'hellooo': 40, 'g': 41, 'morining': 42, 'gmorning': 43, 'afternoon': 44, 'greeting': 45, 'its': 46, 'seeing': 47, 'g’day': 48}

Training sequences:
 [[4], [9], [10], [11], [5], [6], [2, 12], [2, 5], [13], [14], [15], [16], [17, 18], [19, 20], [21], [22], [23], [24], [25], [26], [2, 27], [28, 29], [30], [31, 32, 33, 34, 7, 8, 3], [35], [36], [37], [4, 38], [39], [40], [41, 42], [43], [2, 44], [2, 6], [45], [2, 7, 8, 3], [46, 2, 47, 3], [48]]

Padded training shape: (38, 7)
Training sequences da

In [91]:
model = OneClassSVM(gamma='scale', nu=0.01)
model.fit(train_padded)

OneClassSVM(nu=0.01)

In [92]:
#+1 for normal examples, so-called inliers, and a -1 for outliers.
yhat = model.predict(test_padded)

In [93]:
yhat

array([ 1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1])

In [94]:
score = f1_score(testy, yhat, pos_label=-1)
print('F1 Score: %.3f' % score)

F1 Score: 1.000


In [95]:
def classify(sent):
  l = [sent]
  test_sequences = tokenizer.texts_to_sequences(l)
  test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
  pred_class = model.predict(test_padded)
  cl = int(*pred_class)
  return cl
  
print(classify('Hey! Lets talk, bot'))

1


# Part II: Question Answering

In [61]:
!pip install datasets

In [62]:
import datasets
import requests

# [ds for ds in datasets.list_datasets() if 'ml' in ds.lower()]

In [63]:
qa_ds = datasets.load_dataset('squad', streaming = False)
qa_ds

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [39]:
qa_ds['train'].description

'Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.\n'

In [40]:
print(qa_ds['train'].dataset_size)
qa_ds['train'].features

89846964


{'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None),
 'context': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None)}

In [41]:
qa_ds['train'].to_pandas().head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


In [42]:
url = 'https://raw.githubusercontent.com/Teasotea/DialogSystem/main/data/ml_info.txt'
url2 = 'https://github.com/jamescalam/transformers/blob/main/data/text/meditations/clean.txt'
text_data = requests.get(url)
text_data

<Response [200]>

In [43]:
text_data_text = text_data.text
print(text_data_text[:200])

Linear Regression is a supervised machine learning algorithm where the predicted output is continuous and has a constant slope. It’s used to predict values within a continuous range, (e.g. sales, pric


## Developing QA Part

In [64]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

In [78]:
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)
# tokenizer3 = AutoTokenizer.from_pretrained(model_name)

In [79]:
# context = 'As a good example of distillation, we have DistilBert, if we compare its performance against: BERT base which is not one but DistilBERT’s teacher we can obviously say that DistilBERT compares surprisingly well to BERT: we are able to retain more than 95% of the performance with 40% fewer parameters. All this while in terms of inference time, DistilBERT is 60% faster and smaller than BERT and 120% faster and smaller than ELMo+BiLSTM'
# qa = str(input('question?'))

question?d


In [80]:
# nlp = pipeline('question-answering', model=model_name, tokenizer3=model_name)
# QA_input = { 
#     'question':qa,
#     'context': context}

In [81]:
# res = nlp(QA_input)
# print(res['answer'])

DistilBert


## Trials with BERT

In [49]:
from transformers import BertTokenizer
b_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [50]:
qa_ds['train'] = qa_ds['train'].map(
    lambda x: b_tokenizer(
        x['question'], x['context'], max_length = 512, padding = 'max_length', truncation = True
    ), batched = True, batch_size = 32
)

  0%|          | 0/2738 [00:00<?, ?ba/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

KeyboardInterrupt: ignored

### Pretraining BERT using MLM and NSP

In [ ]:
import requests
import random
from tqdm import tqdm
from transformers import AdamW
from transformers import BertForPreTraining #for MLM and NSP

In [ ]:
bert_qa_model = BertForPreTraining.from_pretrained('bert-base-uncased')

In [ ]:
url = 'https://raw.githubusercontent.com/Teasotea/DialogSystem/main/data/ml_info.txt'
url2 = 'https://github.com/jamescalam/transformers/blob/main/data/text/meditations/clean.txt'
text_data = requests.get(url)
text_data

In [ ]:
text_data_text = text_data.text.split('\n')
text_data_text[:3]

In [ ]:
bag = [sent for p in text_data_text for sent in p.split(',') if sent != '']
bag_size = len(bag)

In [ ]:
sent_a, sent_b, label = [], [], []
for p in text_data_text:
  sents = [
           sent for sent in p.split('.') if sent != ''
  ]
  num_sent = len(sents)
  if num_sent>1:
    start = random.randint(0, num_sent-2)
    sent_a.append(sents[start])
    if random.random() > 0.5:
      sent_b.append(sents[start+1])
      label.append(0)
    else:
      sent_b.append(bag[random.randint(0, bag_size - 1)])
      label.append(1)
      

In [ ]:
inputs = b_tokenizer(sent_a, sent_b, return_tensors = 'pt', max_length = 512, truncation = True, padding = 'max_length')

In [ ]:
inputs.keys()

In [ ]:
inputs['next_sentence_label'] = torch.LongTensor([label]).T
inputs['next_sentence_label'][:10]

In [ ]:
inputs['label'] = inputs.input_ids.detach().clone()

In [ ]:
rand = torch.rand(inputs.input_ids.shape)
mask_arr = (rand < 0.15) * (inputs.input_ids !=  101) *  (inputs.input_ids !=  102) *  (inputs.input_ids !=  0)

In [ ]:
for i in range(inputs.input_ids.shape[0]):
  sel = torch.flatten(mask_arr[i].nonzero()).tolist()
  inputs.input_ids[i, sel] = 103

In [ ]:
class PrepareDataset(torch.utils.data.Dataset):
  def __init__(self, encodings): 
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
data = PrepareDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(data, batch_size = 16, shuffle = True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
bert_qa_model.to(device)

In [ ]:
bert_qa_model.train()

In [ ]:
optim = AdamW(bert_qa_model.parameters(), lr=5e-5)

In [ ]:
for epoch in range(2):
  loop = tqdm(loader, leave = True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    next_sentence_label = batch['next_sentence_label'].to(device)
    labels = batch['labels'].to(device)
    outputs = bert_qa_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask,next_sentence_label=next_sentence_label)
    loss = outputs.loss
    loss.backward()
    optim.step()
    loop.set_description(f'Epoch{epoch}')
    loop.set_postfix(loss=loss.item())

# Part III: Natural Language Generation

In [96]:
# checkpoint 
checkpoint = "microsoft/DialoGPT-medium"
# download and cache tokenizer
tokenizer2 = AutoTokenizer.from_pretrained(checkpoint)
# download and cache pre-trained model
modelNLG = AutoModelForCausalLM.from_pretrained(checkpoint)

# Part IV: Bot Development

In [97]:
#change the code later, make it better
class ChatBot():
    def __init__(self):
        # once chat starts, the history will be stored for chat continuity
        self.chat_history_ids = None
        # make input ids global to use them anywhere within the object
        self.bot_input_ids = None
        # a flag to check whether to end the conversation
        self.end_chat = False
        # greet while starting
        self.welcome()
        self.is_greeting = False
        self.text_input = None
        
    def welcome(self):
        print("Initializing ChatBot ...")
        # some time to get user ready
        time.sleep(2)
        print('Type "bye" or "quit" or "exit" to end chat \n')
        # give time to read what has been printed
        time.sleep(3)

        
    def user_input(self):
        # receive input from user
        text = str(input("User    >> "))
        # end conversation if user wishes so
        if text.lower().strip() in ['bye', 'quit', 'exit']:
            # turn flag on 
            self.end_chat=True
            # a closing comment
            print('ChatBot >>  See you soon! Bye!')
            time.sleep(1)
            print('\nQuitting ChatBot ...')
        else:
            # continue chat, preprocess input text
            # encode the new user input, add the eos_token and return a tensor in Pytorch
            if classify(text) == -1:
              self.text_input = text
            else: 
              self.is_greeting = True

    def bot_greet(self):
        greeting = np.random.choice([
            "Welcome, I am ChatBot, here for your kind service",
            "Hey, Great day! I am your virtual assistant",
            "Hello, it's my pleasure meeting you",
            "Hi, I am a ChatBot. Let's chat!"
        ])
        print("ChatBot >>  " + greeting)
        self.is_greeting = False

    def bot_response(self):
        self.new_user_input_ids = tokenizer2.encode(self.text_input + tokenizer2.eos_token, \
                                                       return_tensors='pt')
        # append the new user input tokens to the chat history
        # if chat has already begun
        if self.chat_history_ids is not None:
            self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1) 
        else:
            # if first entry, initialize bot_input_ids
            self.bot_input_ids = self.new_user_input_ids
        
        # define the new chat_history_ids based on the preceding chats
        # generated a response while limiting the total chat history to 1000 tokens, 
        self.chat_history_ids = modelNLG.generate(self.bot_input_ids, max_length=1000, \
                                               pad_token_id=tokenizer2.eos_token_id)
            
        # last ouput tokens from bot
        response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], \
                               skip_special_tokens=True)
        # in case, bot fails to answer
        if response == "":
            response = self.random_response()
        # print bot response
        print('ChatBot >>  '+ response)
        
    def random_response(self):
        i = -1
        response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # iterate over history backwards to find the last token
        while response == '':
            i = i-1
            response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # if it is a question, answer suitably
        if response.strip() == '?':
            reply = np.random.choice(["I don't know", 
                                     "I am not sure"])
        # not a question? answer suitably
        else:
            reply = np.random.choice(["Great", 
                                      "Fine. What's up?", 
                                      "Okay"
                                     ])
        return reply

In [ ]:
# build a ChatBot object
bot = ChatBot()
# start chatting
while True:
    # receive user input
    bot.user_input()
    # check whether to end chat
    if bot.end_chat:
        break
    # output bot response
    if bot.is_greeting == False:
      bot.bot_response()  
    else: bot.bot_greet()  

Initializing ChatBot ...
Type "bye" or "quit" or "exit" to end chat 

ChatBot >>  Hello, it's my pleasure meeting you
ChatBot >>  I use it when I want to say hello.
User    >> f
